In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
# path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path="D:\Dev\Projets\ml-keras/art_de_la_guerre.txt"
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 158974
total chars: 62


In [3]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.3, 0.5, 0.8]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
print("ready")

nb sequences: 52978
Vectorization...
ready


In [5]:
from keras.layers import LSTM,GRU,Dropout
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done...')

Build model...
Done...


In [6]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
52978/52978 [==============================] - 46s 869us/step - loss: 2.9409

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "leurs, ni recevoir le moindre secours.
s"
leurs, ni recevoir le moindre secours.
se le de se se le le de se le se de de se pe se de se sous de le se se de pe de se cos le de se de le se ve se le ve se se pes pe se se se le se se le se de se le se se se des le le de de se de se pe pe se ce pe se de ce pe le ve se ve se se de se se pe se pe ve de ce de ve se pe se le de pe se se le pas le ce sore et le se ve se se pe se le se se se pe de se se ce fous le le le ce de le se se le l
----- diversity: 0.3
----- Generating with seed: "leurs, ni recevoir le moindre secours.
s"
leurs, ni recevoir le moindre secours.
se dé le de le pe en se ves de de pous las de le ves de se de ale de sas de ce se en ares en et se se se pes de ve le los de be sa se le le que le se aus ce ; le pe va de le de tes de que se ve pes pe se pes le

C:\Users\MicroOnde\AppData\Roaming\Python\Python36\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in log


as diffices et le fart à veut ; s'il faut les mentres en souven de la prosise, les abiles tout celui qu'il faut à les raurament, on qu'ils ne peut lui sent que tout sort à même vous ne ne muuvait que vous avoir le manques qu'ils est les le muni
Epoch 6/60
52978/52978 [==============================] - 44s 821us/step - loss: 1.4804

----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: " autorité sans bornes (par le mot de pui"
 autorité sans bornes (par le mot de puissent pas de sont au combattre et de l'ennemi de l'avantage de la provinir de leur armée en de la faire de leur combattre de leur charde de la malitage de la matirez les combattre de l'ennemi et de la faire de l'ennemi et de leur armée de faire de la malitage de la provoir de la malitage de l'avantage de sont au combattre de l'ennemi et de l'artition de la faire les pas de l'ennemi et de l'ennemi 
----- diversity: 0.3
----- Generating with seed: " autorité sans bornes (par le mot de pui"
 aut

que j'avais à vous dire sur les différentes de rache, comme que vous ceur la suraire ; et on savera de vous ordre pour comme de vos riutiles à le forte. les compréiesses des côtés ; ils ne leur alors que les occasions que vous apperez de conduite allité et de ce qu'ils vous de victoire d'en perdant, qu'il ne c'est des cisques de troupes entreplement, bien de sorte ; il est pour rincisacher de la victoire de la victoire où l'armée d'inco
Epoch 13/60
52978/52978 [==============================] - 44s 825us/step - loss: 1.3069

----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "ême qu'ils avaient eu d'éviter jusqu'à l"
ême qu'ils avaient eu d'éviter jusqu'à les troupes de la fait de l'art de la proforte de leur donner de la prendre de la plus de plus des contre les proposés de combattez pas les combats et de l'art de son armée pour le faut de l'art de la marche de la propos de la pour le combattre de la plus sanger de vos pours de la peut de l'art de l

tent, s'il n'a pas une connaissance exactement pas pour un contr'aien ce qu'il a à pas sont faire qu'on les attaques de tout en sera mille ne de voyez couvert une pendant de tirer de ses puessent les mittes dans les plus faire comptira vos troupes avec en faites votre armée, les combattres d'un combatter des ordres taix, cachéez de l'ennemi, servichez les parties ; il ne puit de pour les répendes que vous combattez se condraient à épier
Epoch 20/60
52978/52978 [==============================] - 44s 827us/step - loss: 1.2334

----- Generating text after Epoch: 19
----- diversity: 0.2
----- Generating with seed: " la campagne, ce qui sera une source de "
 la campagne, ce qui sera une source de l'ennemi de ce qui se propos de l'ennemi de la conduite de son armée que vous aurez pas les combats de la partie de la parde de l'ennemi de l'ennemi de l'ennemi de ce qui se propos de la partie de son armée en soutenir en précaur que vous avez devoir son armée pour les faire de son sorte que vous a

...

s pas, fuyez le plus promptement qu'il vous aura le règle de tout le bataille, et d'instruit de l'armée et de prend et de pouvoir serez moindre de l'ennemi combat en pourraient mettre pas et des combats de votre présenter de son le prudence des parties de l'ennemi, n'en oloit le trop trouver pas que vous courrez votre armée, si les désorminés que les terres le bonher de l'actions, s'il se provoer l'avecte que ces ennemis que vous ne ren
----- diversity: 0.8
----- Generating with seed: "s pas, fuyez le plus promptement qu'il v"
s pas, fuyez le plus promptement qu'il veuit aura vous faire tous les troupes qui est de tout où les soits de royes d'un préservuel que le grand sous le fermes ététain aux préparages, et les destenin, que vous faisez rien son est détermine.
ce qui est tanté, ne mandez, les avantages, vaisaient des mêmes, si vous n'aimpez de votre alcer en sera livre instruit de combat.
si l'ennemi se conconne dans désordre en que celui d'amiencours les 
Epoch 31/60
52978/52978 [=

désespérez de rien ; c'est dans les occasions ; il faut le faite ne le procurer en faites pour l'en faites tenation des en ennemis ; s'il se crainde de vous entrechacer sur la parte les troupes sont serviraires de la mayeux que vous pourrez puisse avec le premper votre armée contraire les pouvoir que pour vouler avoir consiste ce qu'ils est trouve pas avantage à vous arder sous les lieux de vous à la plus de moit, leur donner toutes les
----- diversity: 0.8
----- Generating with seed: "désespérez de rien ; c'est dans les occa"
désespérez de rien ; c'est dans les occasions dans des tenir et la victoire articiou qui pourrait dons le proforce la caspoix faite pour les lieu cherchement mal que jour une au combat, pers et pas le sent sur les combats lui ses à propos que la plui épender par plus fortes de faites comme pour trouver sipinent un temps armée est trouvent forcé de mal dans toutes les poursuissant que ce qui sont pas encore de payont dans les villes pour
Epoch 38/60
52978/52978 [=

on de la place ne soit pas prolongée au-ded de l'armée que vous arez disposer pas une réparation est crionte.
attaquez qui pourraient encore se rendre pas de la force que vous aurez procurer ses troupes et de l'ennemi, et que vous vez de son ennemi les villages de la s'en diser dans les combats et les bêtres et des secrer et tous centainaient jamais que vous terrez tenez présuntera s'attaquer tous vous soyez jusqu'à la moindre que par l
Epoch 45/60
52978/52978 [==============================] - 44s 829us/step - loss: 1.1081

----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "el est l'art de la stratégie victorieuse"
el est l'art de la stratégie victorieuse, et que vous serez voisinant de la plus entre que l'ennemi et de vous combattre à la propre lieux que vous aurez pas sont de combatter les villes de l'ennemi, si vous avez dessentions de la partie de l'ennemi, et que le combat, et que vous serez pour les précauver de la propre la conduite que les 

ii. dêtent leurs sont bien est de l'ennemi, sans qu'il y a quelque bas, ou sont aussi besoint, désimprez la bord, qui sont des circonstances ; réduitez-vous bien en aucher ; il ne sera engagent des diserres de l'art de son danger est de posseont que du fillonge cabourent pour sont de l'ogdiversion ne saurait quelque ne attendre et dans qu'ils vous préparez pas heure à la ter
Epoch 52/60
52978/52978 [==============================] - 45s 840us/step - loss: 1.0907

----- Generating text after Epoch: 51
----- diversity: 0.2
----- Generating with seed: " n'éprouverez au contraire que les plus "
 n'éprouverez au contraire que les plus est de la conduite qui se prépare de votre prince de l'ennemi.
ii. si vous êtes en sorte que des préparations de l'ennemi.
ii. les princes que vous aurez vous pourre à l'ennemi.
si vous sans des princes, il faut de l'ennemi, et que vous aurez pas de la conduite de la prodince, il faut de la moindre de l'ennemi.
ii. celui qui se prépare à l'abordant de la condu

ller, il sait faire venir. si vous faitez sans forcé ; si celui où il certain les lieux qui est le secher ; qu'il un soupours que vous en précipiter les chebltes ; tout si dans la prudence des seces ne souffrent les ménaques qu'il cent de la respace de tous vous avez général qu'il avoir être à vos préjavessances et par l'ennemi avent fait repluar au contrôtices ou pour vous le trouver prendre la defficité, c'est se cinq et de la parti, 
Epoch 59/60
52978/52978 [==============================] - 45s 854us/step - loss: 1.0751

----- Generating text after Epoch: 58
----- diversity: 0.2
----- Generating with seed: "es auront grossi le fleuve ou la rivière"
es auront grossi le fleuve ou la rivière de l'ennemi, de tout ce qui sont pas de la pouvoir pour sous l'ennemi de l'ennemi, si vous ne serez pas les plus en désordres de ses troupes ; soyez qu'ils ne sont pas les plus faites de la force de la particule, de tout ce qui sera vous propos le prépara de l'ennemi de l'armée en sorte qui de pro

In [22]:
model.save_weights('my_model_weights.h5')



----- Generating text after Epoch: 60
----- diversity: 0.2
----- Generating with seed: "in leurs frontières, et ne manquaient pa"
in leurs frontières, et ne manquaient pas les parties de la moisin de vous souveraient de leur consiste de les combat, et de l'ennemi de vous devez les combat, des combattre si vous voulerez le terrain, vous ne sortez avoir les plus parties de vous aurez de les victoires de la courres, de vous sourez par les combat, et et de leur conduite que les suraient de leur rendre que vous servirez le plus entrez auraient de leur consiste de les c
----- diversity: 0.3
----- Generating with seed: "in leurs frontières, et ne manquaient pa"
in leurs frontières, et ne manquaient pas le faire ; il fait leur paraile le bon tracheller les combat, des villes, des viler et de les combat, des victoires de la siège de ces recholent de leur reter de vous aurez par les direr de la moisi des souveraient de leur servire de vous souveraient de leur servicetement de leur service de le

In [1]:
on_epoch_end(60,"")

NameError: name 'on_epoch_end' is not defined